# 이지어드민

- 수집을 최근 30일로
- 저장은 나중에

In [1]:
import requests
import psycopg2
from datetime import datetime, timedelta
import time
import pandas as pd
import json
from psycopg2.extras import Json

In [2]:
def get_order_info(start_date, end_date, page):


    partner_key = '73c6ac2e3366905793ab35d5a222d572'
    domain_key = '73c6ac2e33669057718abd505d76aa87366d3eed6643997457a0f9a03797aa9f'
   
    # limit = '100'
    

    url = f'http://api2.cloud.ezadmin.co.kr/ezadmin/function.php?partner_key={partner_key}&domain_key={domain_key}&action=get_order_info&date_type=collect_date&start_date={start_date}&end_date={end_date}&limit=1000&page={page}'
    res = requests.get(url)

    if res.status_code == 200:
        return res.json()['data']
    else:
        return ValueError

In [3]:
def save_order_info(cursor, data):

    query = """
        INSERT INTO ez_order (seq, shop_id, shop_name, order_id, order_id_seq, shop_product_id, product_name, options, qty, order_name, order_mobile, order_tel, recv_name, recv_mobile, recv_tel, recv_address, recv_zip, memo, status, order_cs, collect_date, order_date, trans_date, trans_date_pos, supply_price, amount, extra_money, trans_corp, trans_no, trans_who, prepay_price, gift, hold, org_seq, deal_no, cust_id, order_products)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s::jsonb)
        ON CONFLICT (seq) DO UPDATE
        SET shop_id = EXCLUDED.shop_id, 
        shop_name = EXCLUDED.shop_name, 
        order_id = EXCLUDED.order_id, 
        order_id_seq = EXCLUDED.order_id_seq, 
        shop_product_id = EXCLUDED.shop_product_id, 
        product_name = EXCLUDED.product_name, 
        options = EXCLUDED.options, 
        qty = EXCLUDED.qty, 
        order_name = EXCLUDED.order_name,
        order_mobile = EXCLUDED.order_mobile,
        order_tel = EXCLUDED.order_tel,
        recv_name = EXCLUDED.recv_name,
        recv_mobile = EXCLUDED.recv_mobile,
        recv_tel = EXCLUDED.recv_tel,
        recv_address = EXCLUDED.recv_address,
        recv_zip = EXCLUDED.recv_zip,
        memo = EXCLUDED.memo,
        status = EXCLUDED.status,
        order_cs = EXCLUDED.order_cs,
        collect_date = EXCLUDED.collect_date,
        order_date = EXCLUDED.order_date,
        trans_date = EXCLUDED.trans_date,
        trans_date_pos = EXCLUDED.trans_date_pos,
        supply_price = EXCLUDED.supply_price,
        amount = EXCLUDED.amount,
        extra_money = EXCLUDED.extra_money,
        trans_corp = EXCLUDED.trans_corp,
        trans_no = EXCLUDED.trans_no,
        trans_who = EXCLUDED.trans_who,
        prepay_price = EXCLUDED.prepay_price,
        gift = EXCLUDED.gift,
        hold = EXCLUDED.hold,
        org_seq = EXCLUDED.org_seq,
        deal_no = EXCLUDED.deal_no,
        cust_id = EXCLUDED.cust_id,
        order_products = EXCLUDED.order_products
        ;
    """

    for d in data:
        cursor.execute(query, (d['seq'], d['shop_id'], d['shop_name'], d['order_id'],  d['order_id_seq'], d['shop_product_id'],  d['product_name'], d['options'], d['qty'], d['order_name'], d['order_mobile'], d['order_tel'], d['recv_name'], d['recv_mobile'], d['recv_tel'], d['recv_address'], d['recv_zip'], d['memo'], d['status'], d['order_cs'], d['collect_date'], d['order_date'], d['trans_date'], d['trans_date_pos'], d['supply_price'], d['amount'], d['extra_money'], d['trans_corp'], d['trans_no'], d['trans_who'], d['prepay_price'], d['gift'], d['hold'], d['org_seq'], d['deal_no'], d['cust_id'], json.dumps(d['order_products'])))



In [6]:
# start_date = datetime.strptime("2023-05-01", "%Y-%m-%d")
# end_date = datetime.strptime("2023-05-04", "%Y-%m-%d")
start_date = datetime.today() - timedelta(days=30)
end_date = datetime.today()

date_list = []
current_date = start_date

while current_date <= end_date:
    formatted_date = current_date.strftime("%Y-%m-%d")
    date_list.append(formatted_date)
    current_date += timedelta(days=1)

In [5]:
conn = psycopg2.connect(host='115.68.228.168', user='postgres', password='cldngks', dbname='test', port=5432)
cursor = conn.cursor()


for date in date_list:
    page = 1
    data = ['init']
    while len(data) > 0:
        data = get_order_info(date, date, str(page))
        save_order_info(cursor, data)
        print(len(data), date, page)
        page += 1
 

conn.commit()
conn.close()

0 2023-05-01 1
1000 2023-05-02 1
31 2023-05-02 2
0 2023-05-02 3
399 2023-05-03 1
0 2023-05-03 2
325 2023-05-04 1
0 2023-05-04 2


# 네이버

- 테스트로 테이블을 하나 만들어서 데이터 검증 필요
    - 2023 한달 단위로 비교
- 최근 30일로 만들기

In [10]:
import requests
import bcrypt
import pybase64
import time
import urllib.request
import urllib.parse
from datetime import datetime, timedelta


def get_token(client_id, client_secret, type_="SELF") -> str:
    timestamp = str(int((time.time()-3) * 1000))
    pwd = f'{client_id}_{timestamp}'
    hashed = bcrypt.hashpw(pwd.encode('utf-8'), client_secret.encode('utf-8'))
    client_secret_sign = pybase64.standard_b64encode(hashed).decode('utf-8')

    headers = {"content-type": "application/x-www-form-urlencoded"}
    data_ = {
        "client_id": client_id,
        "timestamp": timestamp,
        "client_secret_sign": client_secret_sign,
        "grant_type": "client_credentials",
        "type": type_
    }

    query = urllib.parse.urlencode(data_)
    url = 'https://api.commerce.naver.com/external/v1/oauth2/token?' + query
    res = requests.post(url=url, headers=headers)
    res_data = res.json()

    while True:
        if 'access_token' in res_data:
            token = res_data['access_token']
            return token
        else:
            print(f'[{res_data}] 토큰 요청 실패')
            time.sleep(1)


def get_new_order_list(token):
    
    headers = {'Authorization': token}
    url = 'https://api.commerce.naver.com/external/v1/pay-order/seller/product-orders/last-changed-statuses'
    
    now = datetime.now()
    before_date = now - timedelta(days=2) #이틀전
    iosFormat = before_date.astimezone().isoformat()

    params = {
            'lastChangedFrom' : iosFormat, #조회시작일시
        }

    res = requests.get(url=url, headers=headers, params=params)
    res_data = res.json()

    if 'data' not in res_data: #조회된 정보가 없을 경우 data키 없음
        print('주문 내역 없음')
        return False

    data_list = res_data['data']['lastChangeStatuses']

    for data in data_list:
        print(data) #주문 정보

    return data_list



In [147]:
import requests
import bcrypt
import pybase64
import time
import urllib.request
import urllib.parse
from datetime import datetime, timedelta


def get_token(client_id, client_secret, type_="SELF") -> str:
    timestamp = str(int((time.time()-3) * 1000))
    pwd = f'{client_id}_{timestamp}'
    hashed = bcrypt.hashpw(pwd.encode('utf-8'), client_secret.encode('utf-8'))
    client_secret_sign = pybase64.standard_b64encode(hashed).decode('utf-8')

    headers = {"content-type": "application/x-www-form-urlencoded"}
    data_ = {
        "client_id": client_id,
        "timestamp": timestamp,
        "client_secret_sign": client_secret_sign,
        "grant_type": "client_credentials",
        "type": type_
    }

    query = urllib.parse.urlencode(data_)
    url = 'https://api.commerce.naver.com/external/v1/oauth2/token?' + query
    res = requests.post(url=url, headers=headers)
    res_data = res.json()

    while True:
        if 'access_token' in res_data:
            token = res_data['access_token']
            return token
        else:
            print(f'[{res_data}] 토큰 요청 실패')
            time.sleep(1)


def get_product_orders(token, lastChangedFrom, moreSequence):
    
    headers = {'Authorization': token}
    url = 'https://api.commerce.naver.com/external/v1/pay-order/seller/product-orders/last-changed-statuses'
   
    params = {
            'lastChangedFrom' : lastChangedFrom, #조회시작일시
            'moreSequence' : moreSequence,
        }

    res = requests.get(url=url, headers=headers, params=params)
    res_data = res.json()

    if 'data' not in res_data: #조회된 정보가 없을 경우 data키 없음
        print('주문 내역 없음')
        return False

    return res_data

In [122]:
# 함수를 2개 만들어서
# 최근 30일 동안 일자별로 호출하고 (기본값 사용)
# 만약 more가 있는 경우 2번째 함수 호출하게끔 하자.
# more가 없을 때까지 반복 시키고.
# 그리고 그걸 db에 저장을 하자.

In [143]:
token = get_token(client_id='6rmQh0T5jWd9KuJeHIlzU0', client_secret='$2a$04$KJqyFL0J3K5XJdxuTkKZFu')
print(f'발급된 토큰 : ', token)


for date in date_list:
    data = get_product_orders(token, date, None)
    print(data['data']['count'])
    time.sleep(3)

    while 'more' in data['data'].keys():
        moreFrom = data['data']['more']['moreFrom']
        moreSequence = data['data']['more']['moreSequence']
        data = get_product_orders(token, moreFrom, moreSequence)
        print(data['data']['count'])
        time.sleep(3) # 나중엔 5초로 변경



발급된 토큰 :  6pkkMpMlGTSvEAAFHHJKCr
15
300
149
214
212
263
291
275
253
285
146
186
149
164
184
174
113
120
167
227
179


KeyboardInterrupt: 

In [149]:
import pandas as pd

In [150]:
df = pd.DataFrame(data['data']['lastChangeStatuses'])

In [151]:
df

,lastChangedDate,productOrderStatus,orderId,productOrderId,claimType,paymentDate,claimStatus,receiverAddressChanged,lastChangedType,giftReceivingStatus
0,2023-06-14T17:32:55.0+09:00,CANCELED,2023061494902171,2023061413414151,CANCEL,2023-06-14T17:16:36.0+09:00,CANCEL_DONE,False,CLAIM_COMPLETED,NaN
1,2023-06-14T18:18:52.0+09:00,PURCHASE_DECIDED,2023061359146921,2023061345844171,NaN,2023-06-13T10:40:57.0+09:00,NaN,False,PURCHASE_DECIDED,NaN
2,2023-06-14T18:20:32.0+09:00,PURCHASE_DECIDED,2023060962428531,2023060987458171,NaN,2023-06-09T06:14:16.0+09:00,NaN,False,PURCHASE_DECIDED,NaN
3,2023-06-14T18:31:29.0+09:00,PURCHASE_DECIDED,2023061358360501,2023061344536661,NaN,2023-06-13T10:13:06.0+09:00,NaN,False,PURCHASE_DECIDED,NaN
4,2023-06-14T18:32:45.0+09:00,PURCHASE_DECIDED,2023060977810191,2023060921962151,NaN,2023-06-09T20:08:30.0+09:00,NaN,False,PURCHASE_DECIDED,NaN
...,...,...,...,...,...,...,...,...,...,...
174,2023-06-15T16:30:41.0+09:00,CANCELED,2023061528133791,2023061550643901,CANCEL,2023-06-15T16:25:30.0+09:00,CANCEL_DONE,False,CLAIM_COMPLETED,NaN
175,2023-06-15T16:57:16.0+09:00,PURCHASE_DECIDED,2023060681528721,2023060662243601,NaN,2023-06-06T02:07:20.0+09:00,NaN,False,PURCHASE_DECIDED,NaN
176,2023-06-15T17:12:14.0+09:00,PURCHASE_DECIDED,2023061231890091,2023061291759361,NaN,2023-06-12T10:20:00.0+09:00,NaN,False,PURCHASE_DECIDED,NaN
177,2023-06-15T17:23:25.0+09:00,PURCHASE_DECIDED,2023061092288101,2023061044422421,NaN,2023-06-10T18:43:55.0+09:00,NaN,False,PURCHASE_DECIDED,NaN


In [140]:
test

{'timestamp': '2023-07-03T18:23:56.689+09:00',
 'data': {'lastChangeStatuses': [{'productOrderId': '2023070272584581',
    'orderId': '2023070221362011',
    'productOrderStatus': 'DELIVERING',
    'paymentDate': '2023-07-02T09:13:17.0+09:00',
    'lastChangedDate': '2023-07-02T17:32:13.0+09:00',
    'receiverAddressChanged': False,
    'lastChangedType': 'DISPATCHED'},
   {'productOrderId': '2023063025753031',
    'orderId': '2023063081845981',
    'productOrderStatus': 'PURCHASE_DECIDED',
    'paymentDate': '2023-06-30T15:30:12.0+09:00',
    'lastChangedDate': '2023-07-02T17:35:12.0+09:00',
    'receiverAddressChanged': False,
    'lastChangedType': 'PURCHASE_DECIDED'},
   {'productOrderId': '2023062595894961',
    'orderId': '2023062547150191',
    'productOrderStatus': 'PURCHASE_DECIDED',
    'paymentDate': '2023-06-25T07:38:19.0+09:00',
    'lastChangedDate': '2023-07-02T17:35:28.0+09:00',
    'receiverAddressChanged': False,
    'lastChangedType': 'PURCHASE_DECIDED'},
   {'produc

In [75]:
def get_more(token, lastChangedFrom):
    
    headers = {'Authorization': token}
    url = 'https://api.commerce.naver.com/external/v1/pay-order/seller/product-orders/last-changed-statuses'

    params = {
            'lastChangedFrom' : lastChangedFrom, #조회시작일시
        }

    res = requests.get(url=url, headers=headers, params=params)
    res_data = res.json()

    if 'data' not in res_data: #조회된 정보가 없을 경우 data키 없음
        print('주문 내역 없음')
        return False

    data_list = res_data['data']['lastChangeStatuses']

    # for data in data_list:
    #     print(data) #주문 정보

    return res_data

datetime.datetime

In [58]:
data['data']['count']

263

In [98]:
date_list[-99]

'2023-07-02T17:31:26.217611+09:00'

In [44]:
def get_order_detail():
    import datetime

    headers = {'Authorization': token}
    url = 'https://api.commerce.naver.com/external/v1/pay-order/seller/product-orders/query'
    
    params = {
        'productOrderIds' : df['productOrderId'].to_list()
    }

    res = requests.post(url=url, headers=headers, json=params)
    res_data = res.json()
    
    # print(res_data)
    if 'data' not in res_data:
        return False
    
    for data in res_data['data']:
        for d in data.keys():
            for d2 in data[d].keys():
                print(f'{d2} : {data[d][d2]}')


get_order_detail()

expectedDeliveryMethod : DELIVERY
takingAddress : {'baseAddress': '경기도 김포시 통진읍', 'zipCode': '10016', 'addressType': 'DOMESTIC', 'detailedAddress': '검암2로 68 코린트', 'tel1': '02-3431-9617', 'tel2': '010-2103-1181', 'isRoadNameAddress': True, 'name': '코린트 반품'}
commissionRatingType : 결제수수료
commissionPrePayStatus : GENERAL_PRD
paymentCommission : 5768
expectedSettlementAmount : 149954
inflowPath : 검색>쇼핑검색(네이버쇼핑)
channelCommission : 0
productImediateDiscountAmount : 197100
productProductDiscountAmount : 1000
sellerBurdenImediateDiscountAmount : 197100
sellerBurdenProductDiscountAmount : 1000
knowledgeShoppingSellingInterlockCommission : 3178
quantity : 1
mallId : ncp_1nrk6s_01
productOrderId : 2023053159736641
productClass : 단일상품
productOrderStatus : PURCHASE_DECIDED
productName : [2+1] 써큐시안 블러드케어 홍국 콜레스테롤 혈행개선에 도움 혈압 복합기능성 3구 선물세트
productId : 7362546798
itemNo : 7362546798
placeOrderStatus : OK
decisionDate : 2023-06-11T01:38:35.0+09:00
unitPrice : 357000
optionPrice : 0
productDiscountAmount

# 쿠팡

In [ ]:
import os
import time
import hmac, hashlib
import urllib.parse
import urllib.request
import ssl
import json
import pandas as pd
import datetime

def 쿠팡_주문수집(createdAtFrom, createdAtTo):
    

    os.environ['TZ'] = 'GMT+0'
    date=(datetime.datetime.now() - datetime.timedelta(hours = 9)).strftime('%y%m%dT%H%M%SZ')
    method = "GET"
    print(date)
    path = f"/v2/providers/openapi/apis/api/v4/vendors/A00197911/ordersheets"
    query = urllib.parse.urlencode({"createdAtFrom": createdAtFrom, "createdAtTo": createdAtTo, "status": "ACCEPT"})
    message = date+method+path+query


    accesskey = "73814036-d7f4-4d56-a254-35908249a0c6"
    secretkey = "86ecbb8dcd4f4d9c4bd83f724ed4bfb4919ba0cc"

    #********************************************************#
    #authorize, demonstrate how to generate hmac signature here
    signature=hmac.new(secretkey.encode('utf-8'),message.encode('utf-8'),hashlib.sha256).hexdigest()
    authorization  = "CEA algorithm=HmacSHA256, access-key="+accesskey+", signed-date="+date+", signature="+signature
    #print out the hmac key
    #print(authorization)
    #********************************************************#

    # ************* SEND THE REQUEST *************
    url = "https://api-gateway.coupang.com"+path+"?%s" % query

    req = urllib.request.Request(url)

    req.add_header("Content-type","application/json;charset=UTF-8")
    req.add_header("Authorization",authorization)

    req.get_method = lambda: method

    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE

    try:
        resp = urllib.request.urlopen(req,context=ctx)
    except urllib.request.HTTPError as e:
        print(e.code)
        print(e.reason)
        print(e.fp.read())
    except urllib.request.URLError as e:
        print(e.errno)
        print(e.reason)
        print(e.fp.read())
    else:
        # 200
        body = resp.read().decode(resp.headers.get_content_charset())
        return body
   

In [ ]:
시작일 = datetime.datetime.today().strftime("%Y-%m-%d") # 오늘날짜
종료일 = datetime.datetime.today().strftime("%Y-%m-%d") # 오늘날짜


test = 쿠팡_주문수집(시작일, 종료일)
tt = json.loads(test)